# Workshop-Notebook: Manuell vs. KI (BirdNET) – NRW
Dieses Notebook ist für einen Workshop. Wir gehen **Schritt für Schritt** durch.

**Das machen wir:**
1. BirdNET und wichtige Pakete installieren
2. Zwei Audio-Dateien laden (Gruppe A + Gruppe B)
3. Optional: NRW-Artenliste laden (zum Filtern)
4. BirdNET laufen lassen und **CSV mit Confidence** speichern
5. Einfache Plots machen
6. Gruppe A und B vergleichen
7. Optional: Sonogramm (Spektrogramm) zum Üben

---
## Dateien, die du nutzen kannst
- `GroupA_mix.wav` (oder mp3)
- `GroupB_mix.wav` (oder mp3)
- `Arten_NRW.txt` (eine wissenschaftliche Art pro Zeile) – optional


## 1) Installation
Diese Zelle nur **einmal** ausführen (pro Notebook/Kernel).

In [ ]:
%pip -q install birdnet pandas numpy matplotlib librosa soundfile ipywidgets


## 2) Imports + kleine Hilfsfunktionen
Das brauchen wir für Analyse, Tabellen und Plots.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import birdnet
import librosa
import librosa.display

def ensure_path(p):
    p = Path(p).expanduser()
    if not p.exists():
        raise FileNotFoundError(f"Nicht gefunden: {p}")
    return p

def species_split(species_name: str):
    # BirdNET nutzt oft: "Genus species_Common Name"
    if "_" in species_name:
        sci, common = species_name.split("_", 1)
        return sci.strip(), common.strip()
    return species_name.strip(), ""

def time_to_seconds(t):
    # BirdNET Zeit-Strings: z.B. '00:00:03.00'
    parts = str(t).split(":")
    parts = [float(x) for x in parts]
    if len(parts) == 3:
        return parts[0]*3600 + parts[1]*60 + parts[2]
    if len(parts) == 2:
        return parts[0]*60 + parts[1]
    return float(parts[0])


## 3) Pfade setzen (Audio + Artenliste)
Lege deine Dateien am besten in den gleichen Ordner wie dieses Notebook.

**Hinweis (macOS):** Wenn MP3 Probleme macht, nutze WAV oder installiere `libsndfile` und `ffmpeg`:
`brew install libsndfile ffmpeg`

In [ ]:
# --- HIER ANPASSEN ---
GROUP_A_AUDIO = r"./GroupA_mix.wav"   # oder .mp3
GROUP_B_AUDIO = r"./GroupB_mix.wav"   # oder .mp3

# Optional: eine wissenschaftliche Art pro Zeile (z.B. "Turdus merula")
NRW_SPECIES_LIST = r"./Arten_NRW.txt"  # auf None setzen, wenn du nicht filtern willst

# Ausgabe-Ordner
OUT_DIR = Path("./workshop_outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)

a_path = ensure_path(GROUP_A_AUDIO)
b_path = ensure_path(GROUP_B_AUDIO)

species_list_path = None
if NRW_SPECIES_LIST is not None:
    p = Path(NRW_SPECIES_LIST)
    if p.exists():
        species_list_path = str(p)
    else:
        print(f"Artenliste nicht gefunden -> ohne Filter weiter: {p}")


## 4) BirdNET Modell laden
Beim ersten Mal lädt BirdNET das Modell automatisch herunter.

In [ ]:
# Akustik-Modell v2.4, TensorFlow
model = birdnet.load("acoustic", "2.4", "tf")


## 5) BirdNET laufen lassen (Gruppe A + Gruppe B)
Wir bekommen pro Zeitfenster eine Vorhersage mit **Confidence**.

Wenn du eine Artenliste gibst (`custom_species_list`), dann zeigt BirdNET nur diese Arten.

In [ ]:
def run_birdnet(audio_path: Path, label: str, custom_species_list: str | None):
    preds = model.predict(
        str(audio_path),
        custom_species_list=custom_species_list
    )
    preds = preds.copy()
    preds["file_label"] = label
    preds["start_sec"] = preds["start_time"].apply(time_to_seconds)
    preds["end_sec"] = preds["end_time"].apply(time_to_seconds)
    preds[["scientific", "common"]] = preds["species_name"].apply(lambda s: pd.Series(species_split(s)))
    return preds

preds_a = run_birdnet(a_path, "GruppeA", species_list_path)
preds_b = run_birdnet(b_path, "GruppeB", species_list_path)

preds_all = pd.concat([preds_a, preds_b], ignore_index=True)
preds_all.head()


## 6) CSV speichern
Es werden 3 Dateien gespeichert:
- `birdnet_predictions_all.csv`
- `birdnet_predictions_GroupA.csv`
- `birdnet_predictions_GroupB.csv`

In [ ]:
out_all = OUT_DIR / "birdnet_predictions_all.csv"
out_a   = OUT_DIR / "birdnet_predictions_GroupA.csv"
out_b   = OUT_DIR / "birdnet_predictions_GroupB.csv"

preds_all.to_csv(out_all, index=False)
preds_a.to_csv(out_a, index=False)
preds_b.to_csv(out_b, index=False)

print("Gespeichert:", out_all)
print("Gespeichert:", out_a)
print("Gespeichert:", out_b)


## 7) Einfache Plots
### 7.1 Top-Arten (maximale Confidence pro Datei)

In [ ]:
def top_species_plot(df: pd.DataFrame, title: str, top_n: int = 12):
    g = (df.groupby("species_name")["confidence"]
           .max()
           .sort_values(ascending=False)
           .head(top_n))
    plt.figure()
    g.sort_values().plot(kind="barh")
    plt.title(title)
    plt.xlabel("Max Confidence")
    plt.tight_layout()
    plt.show()

top_species_plot(preds_a, "Gruppe A – Top-Arten (max Confidence)")
top_species_plot(preds_b, "Gruppe B – Top-Arten (max Confidence)")


### 7.2 Verteilung der Confidence (Histogramm)

In [ ]:
def confidence_hist(df: pd.DataFrame, title: str):
    plt.figure()
    plt.hist(df["confidence"].values, bins=30)
    plt.title(title)
    plt.xlabel("Confidence")
    plt.ylabel("Anzahl")
    plt.tight_layout()
    plt.show()

confidence_hist(preds_a, "Gruppe A – Confidence Histogramm")
confidence_hist(preds_b, "Gruppe B – Confidence Histogramm")


## 8) Vergleich: Welche Arten kommen in A und B vor?
Wir nehmen einen Confidence-Schwellenwert (z.B. 0.25).
Dann vergleichen wir: nur A, nur B, beide.

In [ ]:
THRESH = 0.25  # im Workshop diskutieren und anpassen

set_a = set(preds_a.loc[preds_a["confidence"] >= THRESH, "species_name"].unique())
set_b = set(preds_b.loc[preds_b["confidence"] >= THRESH, "species_name"].unique())

only_a = sorted(set_a - set_b)
only_b = sorted(set_b - set_a)
both   = sorted(set_a & set_b)

print("Schwelle:", THRESH)
print("Nur in Gruppe A:", len(only_a))
print("Nur in Gruppe B:", len(only_b))
print("In beiden:", len(both))

pd.DataFrame({"species_name": only_a}).to_csv(OUT_DIR / "only_in_GroupA.csv", index=False)
pd.DataFrame({"species_name": only_b}).to_csv(OUT_DIR / "only_in_GroupB.csv", index=False)
pd.DataFrame({"species_name": both}).to_csv(OUT_DIR / "in_both.csv", index=False)


## 9) Optional: Wenn euer Mix aus 8 Clips besteht (je 25s) + 1s Pause
Dann können wir jede Vorhersage einer **Clip-Nummer** zuordnen.
Das ist gut für die Auswertung im Workshop (pro Clip).

In [ ]:
CLIP_LEN = 25
GAP_LEN = 1
BLOCK = CLIP_LEN + GAP_LEN

def add_clip_index(df: pd.DataFrame):
    df = df.copy()
    df["clip_idx"] = (df["start_sec"] // BLOCK).astype(int) + 1
    within_clip = (df["start_sec"] % BLOCK) < CLIP_LEN
    return df[within_clip].copy()

preds_a_clip = add_clip_index(preds_a)
preds_b_clip = add_clip_index(preds_b)

# Pro Clip: beste Vorhersage (höchste Confidence)
top_a_by_clip = (preds_a_clip.sort_values("confidence", ascending=False)
                 .groupby("clip_idx", as_index=False)
                 .first()[["clip_idx","species_name","confidence","scientific","common"]])

top_b_by_clip = (preds_b_clip.sort_values("confidence", ascending=False)
                 .groupby("clip_idx", as_index=False)
                 .first()[["clip_idx","species_name","confidence","scientific","common"]])

top_a_by_clip.to_csv(OUT_DIR / "GroupA_top_prediction_per_clip.csv", index=False)
top_b_by_clip.to_csv(OUT_DIR / "GroupB_top_prediction_per_clip.csv", index=False)

top_a_by_clip, top_b_by_clip


## 10) Optional: Sonogramm (Spektrogramm) zum Üben
Wir wählen ein kurzes Zeitfenster und zeichnen das Spektrogramm.
Dann kann man Hören + Bild zusammen nutzen.

In [ ]:
AUDIO_FOR_SONOGRAM = a_path  # auf b_path setzen, wenn du willst
START_SEC = 0
DURATION_SEC = 12

y, sr = librosa.load(str(AUDIO_FOR_SONOGRAM), sr=None, offset=START_SEC, duration=DURATION_SEC)

S = librosa.stft(y, n_fft=2048, hop_length=256)
S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

plt.figure(figsize=(10, 4))
librosa.display.specshow(S_db, sr=sr, hop_length=256, x_axis="time", y_axis="hz")
plt.title(f"Spektrogramm: {AUDIO_FOR_SONOGRAM.name} (t={START_SEC}s..{START_SEC+DURATION_SEC}s)")
plt.colorbar(format="%+2.0f dB")
plt.tight_layout()
plt.show()
